In [ ]:
%pip install nltk
%pip install tweepy
%pip install pandas

In [ ]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
import pandas as pd
import json
import csv
import sys
import time
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ckey = 'o7qidRODRWm5u07PtAWrCpSqf'
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV'
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc'
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret, 'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])

api = tweepy.API(auth)

In [ ]:
def TweetTemizle(Dataset):
    print ("Tweetler Temizleniyor...")
    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[a-zA-Z]+|$[a-zA-Z]+|@[a-zA-Z]+|[,.^_$*%-;鶯!?:]')
    for i in range(len(Dataset["Tweet"])):
        Dataset["Tweet"][i] = pattern.sub('', Dataset["Tweet"][i])
    print ("Tweetler Temizlendi!")
    return Dataset

In [ ]:
def toDataFrame(tweets):
    print ("Veriler Kayıt Ediliyor...")
    # COnvert to data frame
    DataSet = pd.DataFrame()
    
    DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
    DataSet['Tweet'] = [tweet.text for tweet in tweets]
    DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
    DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
    DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
    DataSet['Dil'] = [tweet.lang for tweet in tweets]
    
    """"
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets] 
    users_retweeted = []    
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    
    DataSet['TweetPlace'] = [i for i in tweets_place]
    DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]
    """
    print ("Veriler Kayıt Edildi!")
    return DataSet

In [ ]:
if (not api):
    print ("Bağlantı Yapılamadı!")
    sys.exit(-1)
else:
    print ("Bağlantı Yapıldı!")
    
    aranan=input("Aranacak Kelimeyi Girin: ")
    
    print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
    
    cursor = tweepy.Cursor(api.search_tweets, q=aranan, result_type="new", geocode="55.0000,4.0000,1000km",lang='tr',count=100)
    
    results=[]
    for item in cursor.items(1000): # Remove the limit to 1000
            results.append(item)
            
    print ("Kelimeye Uygun Tweetler Sorgulandı!")
    
    DataSet = toDataFrame(results)
    DataSet.to_csv('tw.csv',index=False)

In [ ]:
Veri=pd.read_csv('tw.csv')
TweetTemizle(Veri)

Veri.head()

In [ ]:
Veri.tail()

In [ ]:
Veri.info()